In [1]:
import gc
import torch
import pandas as pd
import utils.constants as const

from utils.data_loader import get_data_loaders
from utils.xception import Mos_Xception
from openmax.openmax import create_model

/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.20 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
device = torch.device('cuda:0')

In [3]:
for fold in range(1, 6):
    model_path = f'model_weights/fold_{fold}/model.pth'
    data_csv_path = f'data/splits/data_fold_{fold}.csv'
    weibull_path = f'weibull_models/fold_{fold}/weibull_model.pkl'

    df = pd.read_csv(data_csv_path)
    train_dl, _ = get_data_loaders(df)

    net = Mos_Xception(const.NUM_CLASSES)
    net.load_state_dict(torch.load(model_path))
    net.to(device)
    net.eval()

    weibull = create_model(net, train_dl, weibull_path)

    del net
    del weibull
    gc.collect()
    torch.cuda.empty_cache()